In [1]:
pip install datasets


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 521 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 174 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 24.0 MB 9.5 MB/s eta 0:00:01     |█████████████████               | 12.7 MB 6.6 MB/s eta 0:00:02
     |████████████████████████████████| 166 kB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 387 kB 18.7 MB/s eta 0:00:01
     |████████████████████████████████| 11.0 MB 13.1 MB/s eta 0:00:01
     |███████████████████████████

In [3]:
from datasets import load_dataset

dataset = load_dataset("liar")

train_set = dataset['train']
test_set = dataset['test']
validation_set = dataset['validation']

# Print the size of each set
print("Training Set Size:", len(train_set))
print("Test Set Size:", len(test_set))
print("Validation Set Size:", len(validation_set))

# View the first entry in the training set
print(train_set[0])


Training Set Size: 10269
Test Set Size: 1283
Validation Set Size: 1284
{'id': '2635.json', 'label': 0, 'statement': 'Says the Annies List political group supports third-trimester abortions on demand.', 'subject': 'abortion', 'speaker': 'dwayne-bohac', 'job_title': 'State representative', 'state_info': 'Texas', 'party_affiliation': 'republican', 'barely_true_counts': 0.0, 'false_counts': 1.0, 'half_true_counts': 0.0, 'mostly_true_counts': 0.0, 'pants_on_fire_counts': 0.0, 'context': 'a mailer'}
